In [1]:
from sklearn.cluster import KMeans
import numpy as np
import csv
import math
import matplotlib.pyplot
from matplotlib import pyplot as plt

In [2]:
maxAcc = 0.0
maxIter = 0
C_Lambda =[0.001]#0.001,0.002,0.003,0.005,0.01,] #lambda(CHANGE) original is 0.03
TrainingPercent = 80  
ValidationPercent = 10
TestPercent = 10
M = [40]#15,17,20,21,22,23,24,25,26,27,28,30]  #the number of basis function(CHANGE) original is 10,the number of features
PHI = []  #store design metrix
IsSynthetic = False

# Function Definition

In [3]:
def GetTargetVector(filePath):  #generate trainingdata
    t = []
    with open(filePath, 'rU') as f:  #import excel_t(target values)
        reader = csv.reader(f)
        for row in reader:  
            t.append(int(row[0]))  #The first column of datasets represents target values
    #print("Raw Training Generated..")
    return t

def GenerateRawData(filePath, IsSynthetic):  #import excel_X()(69623 datapoints,each datapoint has 46-5=41 features)    
    dataMatrix = [] 
    with open(filePath, 'rU') as fi:
        reader = csv.reader(fi)
        for row in reader:
            dataRow = []
            for column in row:
                dataRow.append(float(column))
            dataMatrix.append(dataRow)   
    
    if IsSynthetic == False :  #np.append:combine dataMatrix and []，axis is not defined，axis=0，add below，1 add right
        dataMatrix = np.delete(dataMatrix, [5,6,7,8,9], axis=1)  #delete subvector 0（entire row）1（entire column）couldn't delete element
    dataMatrix = np.transpose(dataMatrix)  #remove unrelevant datapoints   
    #print ("Data Matrix Generated..")
    return dataMatrix

def GenerateTrainingTarget(rawTraining,TrainingPercent = 80):
    TrainingLen = int(math.ceil(len(rawTraining)*(TrainingPercent*0.01)))  #returns the number which is greater than or equals to parameter
    t           = rawTraining[:TrainingLen]  #
    #print(str(TrainingPercent) + "% Training Target Generated..")
    return t

def GenerateTrainingDataMatrix(rawData, TrainingPercent = 80):
    T_len = int(math.ceil(len(rawData[0])*0.01*TrainingPercent))
    d2 = rawData[:,0:T_len]  #
    #print(str(TrainingPercent) + "% Training Data Generated..")
    return d2

def GenerateValData(rawData, ValPercent, TrainingCount): 
    valSize = int(math.ceil(len(rawData[0])*ValPercent*0.01))
    V_End = TrainingCount + valSize
    dataMatrix = rawData[:,TrainingCount+1:V_End]
    #print (str(ValPercent) + "% Val Data Generated..")  
    return dataMatrix

def GenerateValTargetVector(rawData, ValPercent, TrainingCount): 
    valSize = int(math.ceil(len(rawData)*ValPercent*0.01))
    V_End = TrainingCount + valSize
    t =rawData[TrainingCount+1:V_End]
    #print (str(ValPercent) + "% Val Target Data Generated..")
    return t

def GenerateBigSigma(Data, MuMatrix,TrainingPercent,IsSynthetic):
    BigSigma    = np.zeros((len(Data),len(Data)))  #0 array
    DataT       = np.transpose(Data)
    TrainingLen = math.ceil(len(DataT)*(TrainingPercent*0.01))        
    varVect     = []
    for i in range(0,len(DataT[0])):  #41 dimensions
        vct = []
        for j in range(0,int(TrainingLen)):
            vct.append(Data[i][j])    
        varVect.append(np.var(vct))  #np.var means calculate vct's varience
    
    for j in range(len(Data)):
        BigSigma[j][j] = varVect[j]  #turn varvect to diagnol metrix,covariance matrix
    if IsSynthetic == True:
        BigSigma = np.dot(3,BigSigma)  #multiply
    else:
        BigSigma = np.dot(1,BigSigma)  #(TA said:simultaneously remove this and w_now) or set 1,original is 200
    ##print ("BigSigma Generated..")
    return BigSigma

def GetScalar(DataRow,MuRow, BigSigInv):  
    R = np.subtract(DataRow,MuRow)  
    T = np.dot(BigSigInv,np.transpose(R))  
    L = np.dot(R,T)
    return L

def GetRadialBasisOut(DataRow,MuRow, BigSigInv):    
    phi_x = math.exp(-0.5*GetScalar(DataRow,MuRow,BigSigInv))  #exponential(Gaussian radial basis function)
    return phi_x

def GetPhiMatrix(Data, MuMatrix, BigSigma, TrainingPercent = 80):
    DataT = np.transpose(Data)
    TrainingLen = c(len(DataT)*(TrainingPercent*0.01))         
    PHI = np.zeros((int(TrainingLen),len(MuMatrix))) 
    BigSigInv = np.linalg.inv(BigSigma)  #(scalar)the inverse of sigma
    for  C in range(0,len(MuMatrix)):
        for R in range(0,int(TrainingLen)):
            PHI[R][C] = GetRadialBasisOut(DataT[R], MuMatrix[C], BigSigInv)  #we get equation (2)
    #print ("PHI Generated..")
    return PHI

def GetWeightsClosedForm(PHI, T, Lambda):  #get equation (8)
    Lambda_I = np.identity(len(PHI[0]))
    for i in range(0,len(PHI[0])):
        Lambda_I[i][i] = Lambda
    PHI_T       = np.transpose(PHI)
    PHI_SQR     = np.dot(PHI_T,PHI)
    PHI_SQR_LI  = np.add(Lambda_I,PHI_SQR)  #lambda
    PHI_SQR_INV = np.linalg.inv(PHI_SQR_LI)  #inverse
    INTER       = np.dot(PHI_SQR_INV, PHI_T)  #tansform
    W           = np.dot(INTER, T)  
    ##print ("Training Weights Generated..")
    return W

def GetPhiMatrix(Data, MuMatrix, BigSigma, TrainingPercent = 80):
    DataT = np.transpose(Data)
    TrainingLen = math.ceil(len(DataT)*(TrainingPercent*0.01))         
    PHI = np.zeros((int(TrainingLen),len(MuMatrix))) 
    BigSigInv = np.linalg.inv(BigSigma)
    for  C in range(0,len(MuMatrix)):
        for R in range(0,int(TrainingLen)):
            PHI[R][C] = GetRadialBasisOut(DataT[R], MuMatrix[C], BigSigInv)  #Mumatrix means centroid C
    #print ("PHI Generated..")
    return PHI

def GetValTest(VAL_PHI,W):
    Y = np.dot(W,np.transpose(VAL_PHI))
    ##print ("Test Out Generated..")
    return Y

def GetErms(VAL_TEST_OUT,ValDataAct):
    sum = 0.0
    t=0
    accuracy = 0.0
    counter = 0
    val = 0.0
    for i in range (0,len(VAL_TEST_OUT)):
        sum = sum + math.pow((ValDataAct[i] - VAL_TEST_OUT[i]),2)  #square error
        if(int(np.around(VAL_TEST_OUT[i], 0)) == ValDataAct[i]):
            counter+=1  #correct prediction and increment 
    accuracy = (float((counter*100))/float(len(VAL_TEST_OUT)))
    ##print ("Accuracy Generated..")
    ##print ("Validation E_RMS : " + str(math.sqrt(sum/len(VAL_TEST_OUT))))
    return (str(accuracy) + ',' +  str(math.sqrt(sum/len(VAL_TEST_OUT))))

## Fetch and Prepare Dataset

In [4]:
RawTarget = GetTargetVector('Querylevelnorm_t.csv')
RawData   = GenerateRawData('Querylevelnorm_X.csv',IsSynthetic)

D:\PYTHON\lingshi\lib\site-packages\ipykernel_launcher.py:3: DeprecationWarning: 'U' mode is deprecated
  This is separate from the ipykernel package so we can avoid doing imports until
D:\PYTHON\lingshi\lib\site-packages\ipykernel_launcher.py:12: DeprecationWarning: 'U' mode is deprecated
  if sys.path[0] == '':


## Prepare Training Data

In [5]:
TrainingTarget = np.array(GenerateTrainingTarget(RawTarget,TrainingPercent))
TrainingData   = GenerateTrainingDataMatrix(RawData,TrainingPercent)
print(TrainingTarget.shape)
print(TrainingData.shape)

(55699,)
(41, 55699)


In [6]:
## Prepare Validation Data

In [7]:
ValDataAct = np.array(GenerateValTargetVector(RawTarget,ValidationPercent, (len(TrainingTarget))))
ValData    = GenerateValData(RawData,ValidationPercent, (len(TrainingTarget)))
print(ValDataAct.shape)
print(ValData.shape)

(6962,)
(41, 6962)


In [8]:
## Prepare Test Data

In [9]:
TestDataAct = np.array(GenerateValTargetVector(RawTarget,TestPercent, (len(TrainingTarget)+len(ValDataAct))))
TestData = GenerateValData(RawData,TestPercent, (len(TrainingTarget)+len(ValDataAct)))
print(ValDataAct.shape)
print(ValData.shape)

(6962,)
(41, 6962)


## Closed Form Solution [Finding Weights using Moore- Penrose pseudo- Inverse Matrix]

In [10]:
ErmsArr = []
AccuracyArr = []
for i in range(len(M)):
    kmeans = KMeans(n_clusters=M[i], random_state=0).fit(np.transpose(TrainingData))
    Mu = kmeans.cluster_centers_
    for j in range(len(C_Lambda)):
        BigSigma     = GenerateBigSigma(RawData, Mu, TrainingPercent,IsSynthetic)
        TRAINING_PHI = GetPhiMatrix(RawData, Mu, BigSigma, TrainingPercent)  #design metrix
        W            = GetWeightsClosedForm(TRAINING_PHI,TrainingTarget,(C_Lambda[j])) #(8)
        TEST_PHI     = GetPhiMatrix(TestData, Mu, BigSigma, 100) 
        VAL_PHI      = GetPhiMatrix(ValData, Mu, BigSigma, 100)
        #print(Mu.shape)
        #print(BigSigma.shape)
        #print(TRAINING_PHI.shape)
        #print(W.shape)
        #print(VAL_PHI.shape)
        #print(TEST_PHI.shape)
        TR_TEST_OUT  = GetValTest(TRAINING_PHI,W)  #testingdate
        VAL_TEST_OUT = GetValTest(VAL_PHI,W)
        TEST_OUT     = GetValTest(TEST_PHI,W)

        TrainingAccuracy   = str(GetErms(TR_TEST_OUT,TrainingTarget))
        ValidationAccuracy = str(GetErms(VAL_TEST_OUT,ValDataAct))
        TestAccuracy       = str(GetErms(TEST_OUT,TestDataAct))
       # print ('UBITname      = De Guo')
       # print ('Person Number = 50289395')
       # print ('----------------------------------------------------')
       # print ("------------------LeToR Data------------------------")
       # print ('----------------------------------------------------')
       # print ("-------Closed Form with Radial Basis Function-------")
       # print ('----------------------------------------------------')
        print ("M =  ",M[i],"\nLambda = ",C_Lambda[j])
        print ("Training accuracy  = " + str(float(TrainingAccuracy.split(',')[0])))  #0,accuracy
        print ("Validation accuracy= " + str(float(ValidationAccuracy.split(',')[0])))  
        print ("Testing accuracy   = " + str(float(TestAccuracy.split(',')[0])))  

        print ("E_rms Training   = " + str(float(TrainingAccuracy.split(',')[1])))  #1,error 
        print ("E_rms Validation = " + str(float(ValidationAccuracy.split(',')[1])))  
        print ("E_rms Testing    = " + str(float(TestAccuracy.split(',')[1])))  
        
#plt.plot(M,E_rms Testing)
#plt.plot(C_Lambda,E_rms Testing)

M =   30 
Lambda =  0.03
Training accuracy  = 73.7266378211458
Validation accuracy= 74.34645216891698
Testing accuracy   = 69.08490159459848
E_rms Training   = 0.6251223366105693
E_rms Validation = 0.6121795316539154
E_rms Testing    = 0.7220817684972818


## Gradient Descent solution for Linear Regression

In [11]:
print ('----------------------------------------------------')
print ('--------------Please Wait for 2 mins!----------------')
print ('----------------------------------------------------')

----------------------------------------------------
--------------Please Wait for 2 mins!----------------
----------------------------------------------------


In [12]:
W_Now        = np.dot(1, W)  #set 1,original is 220
La           = 2
learningRate = 0.01  #that can be changed,original one is 0.01 
   
L_Erms_Val   = []  #For each epoch we calculate t,v,t error
L_Erms_TR    = []
L_Erms_Test  = []  #we can do testing after training is done
W_Mat        = []

for i in range(0,400):  #SGD
    Delta_E_D     = -np.dot((TrainingTarget[i] - np.dot(np.transpose(W_Now),TRAINING_PHI[i])),TRAINING_PHI[i])
    La_Delta_E_W  = np.dot(La,W_Now)
    Delta_E       = np.add(Delta_E_D,La_Delta_E_W)  #(10)   
    Delta_W       = -np.dot(learningRate,Delta_E)
    W_T_Next      = W_Now + Delta_W
    #print ('---------Iteration: ' + str(i) + '--------------')
    W_Now         = W_T_Next
    
    #-----------------TrainingData Accuracy---------------------#
    TR_TEST_OUT   = GetValTest(TRAINING_PHI,W_T_Next) 
    Erms_TR       = GetErms(TR_TEST_OUT,TrainingTarget)
    L_Erms_TR.append(float(Erms_TR.split(',')[0]))
    
    #-----------------ValidationData Accuracy---------------------#
    VAL_TEST_OUT  = GetValTest(VAL_PHI,W_T_Next) 
    Erms_Val      = GetErms(VAL_TEST_OUT,ValDataAct)
    L_Erms_Val.append(float(Erms_Val.split(',')[0]))  #change 1 to 0,original is 1
    
    #-----------------TestingData Accuracy---------------------#
    TEST_OUT      = GetValTest(TEST_PHI,W_T_Next) 
    Erms_Test = GetErms(TEST_OUT,TestDataAct)
    L_Erms_Test.append(float(Erms_Test.split(',')[0]))

In [13]:
print ('----------Gradient Descent Solution--------------------')
print ("M = 15 \nLambda  = 0.0001\neta=0.01")
print ("E_rms Training   = " + str(np.around(min(L_Erms_TR),5)))
print ("E_rms Validation = " + str(np.around(min(L_Erms_Val),5)))
print ("E_rms Testing    = " + str(np.around(min(L_Erms_Test),5)))

----------Gradient Descent Solution--------------------
M = 15 
Lambda  = 0.0001
eta=0.01
E_rms Training   = 73.76793
E_rms Validation = 74.43263
E_rms Testing    = 69.0849
